# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [23]:
import pandas as pd
import json
import os
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

load_dotenv()

True

In [24]:
# Import N3 Vocabulary
file_path = '../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()

,expression,reading
0,頂上,ちょうじょう
1,逆,ぎゃく
2,電子,でんし
3,図る,はかる
4,語句,ごく


#### load Models

In [25]:
azure_llm = AzureChatOpenAI(
    azure_endpoint="https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
    api_key=os.environ["AZURE_API_KEY"],
    model_name="gpt-4o",
    api_version="2025-01-01-preview",
    temperature=0.5,
)

In [26]:
# aws_llm = ChatBedrock(
#     model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
#      # model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
#     model_kwargs=dict(temperature=0.3),
#     region = "us-east-2",
#     aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
#     aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
# )

## Question Explore

# Kanji 读假名（读音问题）

In [27]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    improved_question: str
    final_question: str


kanji_example = """
問題 4

つぎの(1)から(4)の文章を読んで、質問に答えなさい。答えは、1・2・3・4から最もよいものを一つえらびなさい。

⸻

(1)

これは、今川さんが後のミゲルさんに書いたメールである。

⸻

ミゲルさん

メールをありがとう。

同じ会社で働くことになって、うれしいです。

住む所についてアドバイスをくださいと書いてあったので、お答えします。

会社まで歩いて行きたいと書いてありましたが、会社の周りはオフィスばかりで、アパートはほとんどありません。電車通勤になりますが、私が以前住んでいた緑野という町はいいですよ。

緑野駅から会社のある北駅まで電車で15分だし、いろいろなお店があって便利です。

いい所が見つかるといいですね。会えるのを楽しみにしています。

今川

⸻

23. まで電車で15分で行けるし、店も多いので、緑野にしたらどうか。
	1.	(選択肢なし)
	2.	いろいろな店があって便利なので、北園駅の近くにしたらどうか
	3.	北駅まで電車で15分で行けるし、店も多いので、緑野にしたらどうか
	4.	いろいろな店があって便利なので、北園駅の近くにしたらどうか
    
(2)

友達のマキは、いいことがあったという話をよくする。だから私は、マキは運がいいのだと思っていた。しかし、最近、そうではないと気づいた。

先日二人で出かけたとき、事故で電車が止まっていて、何キロも歩いて帰ることになった。嫌だなと思っている私に、マキは「知らない町を歩けるね。」と嬉しそうに言った。とても不思議だった。でも、マキは楽しめてしまうのだ。今まで私が聞いた話も、マキだから「いいこと」だと感じたのだろうと思う。

⸻

24. 最近、「私」はマキのことをどのような人だと思うようになったか。
	1.	「いいこと」ばかりが起きる。運がいい人
	2.	「私」と一緒に経験したことは、何でも「いいこと」だと思える人
	3.	ほかの人に起こった「いいこと」を一緒に喜んであげられる人
	4.	ほかの人が「いいこと」だと思わないことも「いいこと」だと思える人

 (3)

(会社で)

ミンさんが席に戻ると、机の上に、原口課長からのメモが置いてあった。

ミンさん

子どもが熱を出したので、早退します。午後、明日の会議の進行について確認する約束だったのに、すみません。午後の話し合いのために予約していた小会議室はキャンセルしてくれますか。席に戻ったら、すぐにお願いします。会議の進行については、明日の朝、最初に確認して、そのあとに会議室の準備をしましょう。

それから、ミンさんの作った資料ですが、問題ないので、今日中に8人分印刷しておいてください。

よろしくお願いします。

9月8日 12:10
原口

⸻

25. このメモを読んで、ミンさんはまず何をしなければならないか。
	1.	会議の進行について口課長と確認する
	2.	小会議室をキャンセルする
	3.	会議室の準備をする
	4.	会議の資料を8人分印刷する

    (4)

日本のファミリーレストランは、店の壁やソファーなどに、赤やオレンジ色のような暖かさを感じさせる色、つまり、暖色を使うことが多い。

暖色には食欲を感じさせる効果があるので、暖色に囲まれていると、料理がおいしそうに見える。また、暖色は、時間を実際より長く感じさせる効果もある。客は、店にいた時間が短くても、ゆっくりできたように感じるのだ。

⸻

26.

日本のファミリーレストランで暖色が使われる理由は何か。
	1.	店の暖房にあまりお金がかからないようにするため
	2.	客に、店の料理と店で過ごす時間にいい印象を持ってもらうため
	3.	店をおしゃれに見せて、客に店に入りたいと思ってもらうため
	4.	客に長く店にいてもらって、料理をたくさん注文してもらうため       
"""

print(words)

# Nodes
def generate_question(state: QuestionState):
    """First LLM call to generate initial question"""

    msg = azure_llm.invoke(f"You are a japanese teacher. Your job is Read 4 short articles in lines 5-6, with a word count of around 200 words, and choose the correct answer. Including some emails Notification and letter articles, answer one question. Only refer to the format, not the content."
            f"The vocabulary should be restricted to N3 level, you can refer to the vocabulary in the word list, choosing random words for the questions"
            f"please refer the question examples following the formal exam paper"
            f"append the correct answer and explanation of main challenges and why teacher asks this question to candidate in chinese at each question"
            f"Finally, beautify markdown format"
            f"topic list: {state['topic']}"
            f"word list: {words}"
            f"formal exam paper: {kanji_example}")
    
    return {"question": msg.content}


# Build workflow
kanji_workflow = StateGraph(QuestionState)

# Add nodes
kanji_workflow.add_node("generate_question", generate_question)

# Add edges to connect nodes
kanji_workflow.add_edge(START, "generate_question")
kanji_workflow.add_edge("generate_question", END)

# Compile
kanji_graph = kanji_workflow.compile()


     expression reading
0            頂上  ちょうじょう
1             逆     ぎゃく
2            電子     でんし
3            図る     はかる
4            語句      ごく
...         ...     ...
2134        超える     こえる
2135        載せる     のせる
2136         進歩     しんぽ
2137          敵      てき
2138          嫁      よめ

[2139 rows x 2 columns]


In [28]:
# Show workflow
#display(Image(kanji_graph.get_graph().draw_png()))

In [29]:
# Invoke
kanji = kanji_graph.invoke({"topic": "店で価格を尋ねる | レストランで食べ物を注文する | 交通手段について話す| 天気の状況について話す | 趣味について話す | 店で商品の特徴や価格を尋ねる | スーパーマーケットで買い物をする | レストランで食べ物や飲み物を注文する | カフェでテイクアウトを頼む | コンビニで支払い方法について質問する |  ネットショッピングで商品を選ぶ"})
display(Markdown(kanji["question"]))

AuthenticationError: Error code: 401 - {'error': {'code': '401', 'message': 'Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.'}}